# Rendering CNVs (old style vs proposed new)
Utilizing current VRS definitions to model a Copy Number Variation using my own examples from http://atlasgeneticsoncology.org/index.html. After established an example or two using current VRS implementation, utilize information from discussion with Alex about proposed new model in which a CNV is another type of DerivedSequenceExpression. Should also contain explainations of everything as going to make sure everything is understood.

In [1]:
# Import VRS
from ga4gh.vrs import models
from ga4gh.vrs.extras.translator import Translator
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy

# Import JSON to dump VRS objects to JSON format
import json
from IPython.display import Image



Removing allOf attribute from CopyNumber to avoid python-jsonschema-objects error.
Removing allOf attribute from SequenceInterval to avoid python-jsonschema-objects error.
Removing allOf attribute from RepeatedSequenceExpression to avoid python-jsonschema-objects error.
/usr/local/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


## Current VRS

### Example 1: Gene has an increased copy number
An example of this will be to model the alpha-amylase gene (AMY1). AMY1 is an enzyme in saliva responsible for breakdown of starch. This gene has a wide range of copy number throughout different human populations. It also has convincing evidence that correlates its protein function to its copy number.

(Source: Wikipedia https://en.wikipedia.org/wiki/Copy_number_variation#Alpha-amylase_gene)

In [2]:
# Set variation number using DefiniteRange VRS object. AMY1's copy number can vary from 2 to 16 so we will define ranges
amy1_defrange = models.DefiniteRange(min=2,max=16)

# With range defined, now represent the copy number variation potential of AMY1 using the VRS object
amy1_cn = models.CopyNumber(copies=amy1_defrange,
                            subject=models.Gene(gene_id='ncbigene:11722')
                            )

In [3]:
# AMY1 copy number variation representation at gene level
amy1_cn

<CopyNumber _id=None copies=<DefiniteRange max=<Literal<int> 16> min=<Literal<int> 2> type=<Literal<str> DefiniteRange>> subject=<Gene gene_id=<Literal<str> ncbigene:11722> type=<Literal<str> Gene>> type=<Literal<str> CopyNumber>>

In [4]:
# JSON format
amy1_json = json.dumps(amy1_cn.as_dict(), indent=1)
print(amy1_json)

{
 "type": "CopyNumber",
 "subject": {
  "type": "Gene",
  "gene_id": "ncbigene:11722"
 },
 "copies": {
  "type": "DefiniteRange",
  "min": 2,
  "max": 16
 }
}


### Example 2: Exon has a duplication event

Coding regions within a gene can be duplicated within the genome due to errors in repair, among other abnormalities. In one study, it was noted that six patients with pilocytic astrocytomas (grade II-III) had a copy number gain in KIAA1549-BRAF gene fusion, or a 7q34 gain. Patients with this gain showed excellent long term survival. 

(source: https://onlinelibrary.wiley.com/doi/10.1002/jcp.21978) 

In [5]:
# First, specify the region being duplicated (KIAA1549 7q34)
kiaa_seqinterval = models.SequenceInterval(start=models.Number(value=138831377),end=models.Number(value=138981626))
kiaa_location = models.SequenceLocation(interval=kiaa_seqinterval, sequence_id='refseq:NC_000007.14')

# Now that we have the region (KIAA1549), define the derived sequence and the gain
kiaa_deriveseq = models.DerivedSequenceExpression(location=kiaa_location, reverse_complement=False)
kiaa_cnv_gain = models.IndefiniteRange(comparator=">=",value=2)

# With the kiaa sequence defined and the cnv gain defined, express as a copy number object
kiaa_copy_gain = models.CopyNumber(copies=kiaa_cnv_gain, subject=kiaa_deriveseq)

In [6]:
# kiaa_dup_json = json.dumps(kiaa_copy_gain.as_dict(), indent=1)
# print(kiaa_dup_json)
print(kiaa_copy_gain.as_dict())

{'type': 'CopyNumber', 'subject': {'type': 'DerivedSequenceExpression', 'location': {'type': 'SequenceLocation', 'sequence_id': 'refseq:NC_000007.14', 'interval': {'type': 'SequenceInterval', 'start': {'type': 'Number', 'value': 138831377}, 'end': {'type': 'Number', 'value': 138981626}}}, 'reverse_complement': False}, 'copies': {'type': 'IndefiniteRange', 'value': 2, 'comparator': '>='}}


## Proposed Change

Copy number representation is not sufficient in current version of VRS. After discussions with Alex, we determined that it was essentially another version of DerivedSequenceExpression. It should have its own representation as another instance of DerivedSequenceExpression.

In [7]:
class DerivedCytobandExpression(): # Parent Class in parantheses...super init if __init__ params diff
    def __init__(self,location,reverse_complement):
        self.type = 'CytobandExpression'
        self.location = location
        self.reverse_complement = reverse_complement
        self.__is_valid_location(location)

    def __is_valid_location(self,location):
        if location.type != 'SequenceLocation':
            raise ValueError("Must contain valid SequenceLocation")
        return location.type

    def __iter__(self):
        yield 'type', self.type
        yield 'location', self.location
        yield 'reverse_complement', self.reverse_complement

    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)


class CopyNumber():
    def __init__(self,copies,cytoband):
        self.type = 'CopyNumber'
        self.copies = copies
        self.cytoband = cytoband

    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)



### Example 2: Exon duplication event (new model)

Coding regions within a gene can be duplicated within the genome due to errors in repair, among other abnormalities. In one study, it was noted that six patients with pilocytic astrocytomas (grade II-III) had a copy number gain in KIAA1549-BRAF gene fusion, or a 7q34 gain. Patients with this gain showed excellent long term survival. 

This time, use new DerivedCytobandExpression class to express this event. Having this separate will also allow for hashing without overlapping DerivedSequenceExpressions or CopyNumber objects.

(source: https://onlinelibrary.wiley.com/doi/10.1002/jcp.21978) 

In [10]:
# First, specify the region being duplicated (KIAA1549 7q34)
kiaa_seqinterval = models.SequenceInterval(start=models.Number(value=138831377),end=models.Number(value=138981626))
kiaa_location = models.SequenceLocation(interval=kiaa_seqinterval, sequence_id='refseq:NC_000007.14')

# Now that we have the region (KIAA1549), define the derived sequence and the gain
kiaa_cytoband_expression = DerivedCytobandExpression(location=kiaa_location, reverse_complement=False)
kiaa_cnv_gain = models.IndefiniteRange(comparator=">=",value=2)

# Validation test
kiaa_incorrect_expression = DerivedCytobandExpression(location='lol',reverse_complement=False)

# f = kiaa_cytoband_expression.toJSON()
# json.dumps(kiaa_cytoband_expression.as_dict(), indent=1)

# print(f)

AttributeError: 'str' object has no attribute 'type'

In [63]:
# From here, we can then use the existing CopyNumber object to represent our copy number change object
kiaa_copy_gain = CopyNumber(copies=kiaa_cnv_gain, cytoband=kiaa_cytoband_expression)

